In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hl
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/exomeQC-hail-gnomad/notebooks/hail-20210617-1059-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [2]:

def get_binned_models_pd(ht: hl.Table, models: Union[Dict[str, str], List[str]], contigs: Set[str] = None) -> pd.DataFrame:
    """
    Creates a single DataFrame with all desired models binned and ready for plotting.

    :param str data_type: One of 'exomes' or 'genomes'
    :param list of str models: Models to load. Either a list of the model ids, or a dict with model id -> model name for display
    :param list of str contigs: Contigs to load
    :return: Plot-ready DataFrame
    :rtype: DataFrame
    """

    def aggregate_contig(ht: hl.Table, contigs: Set[str] = None):
        """
        Aggregates all contigs together and computes number for bins accross the contigs.
        """
        if contigs:
            ht = ht.filter(hl.literal(contigs).contains(ht.contig))

        return ht.group_by(
            *[k for k in ht.key if k != 'contig']
        ).aggregate(
            min_score=hl.agg.min(ht.min_score),
            max_score=hl.agg.max(ht.max_score),
            **{x: hl.agg.sum(ht[x]) for x in ht.row_value if x not in ['min_score', 'max_score']}
        )

    if not isinstance(models, dict):
        models = {m: m for m in models}

    hts = [
        aggregate_contig(ht, contigs)
            .annotate(model=model_name) for model_id, model_name in models.items()
    ]

    ht = hts[0].union(*hts[1:])
    #ht = ht.annotate(n_biallelic=hl.cond(ht.bi_allelic, ht.n, 0))
    return ht.to_pandas()
qc_plots_settings = {
    'mean_point_size': 4.0,
    'min_point_size': 1.0,
    'max_point_size': 16.0,
    'label_text_font_size': "14pt",
    'title.text_font_size': "16pt",
    'subtitle.text_font_size': "14pt",
    'axis.axis_label_text_font_size': "16pt",
    'axis.axis_label_text_font_style': "normal",
    'axis.major_label_text_font_size': "14pt"
}
def set_plots_defaults(p: Plot) -> None:
    p.legend.label_text_font_size = qc_plots_settings['label_text_font_size']
    p.title.text_font_size = qc_plots_settings['title.text_font_size']
    p.axis.axis_label_text_font_size = qc_plots_settings['axis.axis_label_text_font_size']
    p.axis.axis_label_text_font_style = qc_plots_settings['axis.axis_label_text_font_style']
    p.axis.major_label_text_font_size = qc_plots_settings['axis.major_label_text_font_size']
    
def get_point_size_col(data: pd.Series, size_prop: str) -> pd.Series:
    """
    Given a data Series, returns the corresponding point size either:
    - Constant to qc_plots_settings['mean_point_size'] if `size_prop` is None
    - Radius proportional to data, if `size_prop` is 'radius'
    - Area proportional to data, if `size_prop` is 'area'

    Mean, min and max point  sizes are extracted from qc_plots_settings

    :param Series data: Input data series
    :param str size_prop: One of None, 'radius' or 'area'
    :return: Series with corresponding point size for each data point
    :rtype: Series
    """
    if size_prop is None:
        return pd.Series(len(data) * [qc_plots_settings['mean_point_size']])
    else:
        mean_data = np.mean(data)
        if size_prop == 'radius':
            return data.apply(lambda x: max(qc_plots_settings['min_point_size'], min(qc_plots_settings['max_point_size'], qc_plots_settings['mean_point_size'] * (x / mean_data))))
        elif size_prop == 'area':
            return data.apply(lambda x: max(qc_plots_settings['min_point_size'], min(qc_plots_settings['max_point_size'], qc_plots_settings['mean_point_size'] * np.pi * (np.sqrt(x / mean_data) / np.pi))))
        else:
            raise ValueError(f"{size_prop} is not a supported value for argument `size_prop`")
            
def plot_metric(df: pd.DataFrame,
                y_name: str,
                cols: List[str],
                y_fun: Callable[[pd.Series], Union[float, int]] = lambda x: x,
                cut: int = None,
                plot_all: bool = True,
                plot_bi_allelics: bool = True,
                plot_singletons: bool = True,
                plot_bi_allelic_singletons: bool = True,
                plot_adj: bool = False,
                colors: Dict[str, str] = None,
                link_cumul_y: bool = True,
                size_prop: str = 'area'
                ) -> Tabs:
    """
    Generic function for generating QC metric plots using a plotting-ready DataFrame (obtained from `get_binned_models_pd`)
    DataFrame needs to have a `rank_id` column, a `bin` column and a `model` column (contains the model name and needs to be added to binned table(s))

    This function generates scatter plots with the metric bin on x-axis and a user-defined function on the y-axis.
    The data for the y-axis function needs to from the columns specified in `cols`. The function is specified with the `y_fun` argument and data columns are access as a list.
    As an example, plotting Transition to transversion ratio is done as follows::

        plot_metric(snvs, 'Ti/Tv', ['n_ti', 'n_tv'], y_fun=lambda x: x[0]/x[1], colors=colors)

    In this command, `x[0]` correspond to the  first column selected (`'n_ti'`)  and `x[1]` to the second (`'n_tv'`).


    This function plots a tab for each of the plot condition(s) selected: all, bi-allelics, bi-allelic singletons.
    Within each tab, each row contains a non-cumulative and a cumulative plot of the bins / values.
    If `plot_adj` is set, then an extra row is added plotting only variants in release samples where AC_ADJ>0. The bin for these sites is computed based on those variants only.

    :param pd.DataFrame df: Input data
    :param str y_name: Name of the metric plotted on the y-axis
    :param list of str cols: Columns used to compute the metric plotted
    :param callable y_fun: Function to apply to the columns to generate the metric
    :param int cut: Where to draw the bin cut
    :param bool plot_all: Whether to plot a tab with all variants
    :param bool plot_bi_allelics: Whether to plot a tab with bi-allelic variants only
    :param bool plot_singletons: Whether to plot a tab with singleton variants only
    :param bool plot_bi_allelic_singletons:  Whether to plot a tab with bi-allelic singleton variants only
    :param bool plot_adj: Whether to plot additional rows with adj variants in release samples only
    :param dict of str -> str colors: Mapping of model name -> color
    :param bool link_cumul_y: If set, y-axes of cumulative and non-cumulative plots are linked
    :param str size_prop: Either 'size' or 'area' can be specified. If either is specified, the points will be sized proportionally to the amount of data in that point.
    :return: Plot
    :rtype: Tabs
    """

    def get_row(df: pd.DataFrame, y_name: str, cols: List[str], y_fun: Callable[[pd.Series], Union[float, int]], titles: List[str], link_cumul_y: bool, cut: int = None) -> Row:
        """
        Generates a single row with two plots: a regular scatter plot and a cumulative one.
        Both plots have bins on the x-axis. The y-axis is computed by applying the function `y_fun` on the columns `cols`.

        Data source is shared between the two plots so that highlighting / selection is linked.
        X-axis is shared between the two plots.
        Y-axus is shared if `link_cumul_y` is `True`

        """

        def get_plot(data_source: ColumnDataSource, y_name: str, y_col_name: str, titles: List[str], data_ranges: Tuple[DataRange1d, DataRange1d], cut: int = None) -> Plot:
            """
            Generates a single scatter plot panel
            """

            p = figure(
                title=titles[0],
                x_axis_label='bin',
                y_axis_label=y_name,
                tools="save,pan,box_zoom,reset,wheel_zoom,box_select,lasso_select,help,hover")
            p.x_range = data_ranges[0]
            p.y_range = data_ranges[1]

            if cut:
                p.add_layout(Span(location=cut, dimension='height', line_color='red', line_dash='dashed'))

            # Add circles layouts one model at a time, so that no default legend is generated.
            # Because data is in the same ColumnDataSource, use a BooleanFilter to plot each model separately
            circles = []
            for model in set(data_source.data['model']):
                view = CDSView(source=data_source, filters=[BooleanFilter([x == model for x in data_source.data['model']])])
                circles.append((model, [p.circle('bin', y_col_name, color='_color', size='_size', source=data_source, view=view)]))

            p.select_one(HoverTool).tooltips = [('model', '@model'),
                                                ('bin', '@bin'),
                                                (y_name, f'@{y_col_name}'),
                                                ('min_score', '@min_score'),
                                                ('max_score', '@max_score'),
                                                ('n_data_points', '@_n')
                                                ] + [(col, f'@{col}') for col in cols]
            set_plots_defaults(p)

            # Add legend above the plot area
            legend = Legend(items=circles, orientation='horizontal', location=(0, 0), click_policy="hide")
            p.add_layout(legend, 'above')

            # Add subtitles if any
            for title in titles[1:]:
                p.add_layout(Title(text=title, text_font_size=qc_plots_settings['subtitle.text_font_size']), 'above')

            return p

        # Compute non-cumulative values by applying `y_fun`
        df['non_cumul'] = df[cols].apply(y_fun, axis=1)

        # Compute cumulative values for each of the data columns
        for col in cols:
            df[f'{col}_cumul'] = df.groupby('model').aggregate(np.cumsum)[col]
        df['cumul'] = df[[f'{col}_cumul' for col in cols]].apply(y_fun, axis=1)

        # Create data ranges that are either shared or distinct depending on the y_cumul parameter
        non_cumul_data_ranges = (DataRange1d(), DataRange1d())
        cumul_data_ranges = non_cumul_data_ranges if link_cumul_y else (non_cumul_data_ranges[0], DataRange1d())
        data_source = ColumnDataSource(df)

        return Row(get_plot(data_source, y_name, 'non_cumul', titles, non_cumul_data_ranges, cut),
                   get_plot(data_source, y_name, 'cumul', [titles[0] + ', cumulative'] + titles[1:], cumul_data_ranges, cut))

    def prepare_pd(df: pd.DataFrame, cols: List[str], colors: Dict[str, str] = {}, size_prop: str = None):
        """
        Groups a pandas DataFrame by model and bin while keeping relevant columns only.
        Adds 3 columns used for plotting:
        1. A _color column column
        2. A _n column containing the number of data points
        3. A _size column containing the size of data points based on the `size_prop` and `qc_plot_settings` parameters
        """
        df = df.groupby(['model', 'bin']).agg({**{col: np.sum for col in cols},
                                               'min_score': np.min, 'max_score': np.max})
        df = df.reset_index()
        df['_color'] = [colors.get(x, 'gray') for x in df['model']]
        df['_n'] = np.sum(df[cols], axis=1)
        df['_size'] = get_point_size_col(df['_n'], size_prop)
        return df

    colors = colors if colors is not None else {}
    tabs = []
    adj_strats = ['', 'adj_'] if plot_adj else ['']

    if plot_all:
        children = []
        for adj in adj_strats:
            titles = [y_name, 'Adj variants (adj rank)' if adj else 'All variants']
            plot_df = prepare_pd(df.loc[df.rank_id == f'{adj}rank'], cols, colors, size_prop)
            if len(plot_df) > 0:
                children.append(get_row(plot_df, y_name, cols, y_fun, titles, link_cumul_y, cut))
            else:
                logger.warn('No data found for plot: {}'.format('\t'.join(titles)))

        if children:
            tabs.append(Panel(child=Column(children=children), title='All'))

   
    if plot_singletons:
        children = []
        for adj in adj_strats:
            for singleton_rank in ['', 'singleton_']:
                titles = [y_name, 'Singletons ({} rank)'.format('overall' if not adj and not singleton_rank else " ".join([adj[:-1], singleton_rank[:-1]]).lstrip())]
                plot_df = prepare_pd(df.loc[df.singleton & (df.rank_id == f'{adj}{singleton_rank}rank')], cols, colors, size_prop)
                if len(plot_df) > 0:
                    children.append(get_row(plot_df, y_name, cols, y_fun, titles, link_cumul_y, cut))
                else:
                    logger.warn('No data found for plot: {}'.format('\t'.join(titles)))

        if children:
            tabs.append(Panel(child=Column(children=children), title='Singletons'))

   
    return Tabs(tabs=tabs)

In [3]:
run_hash = "ae281191"
ht = hl.read_table(f'{lustre_dir}/variant_qc/models/{run_hash}_rf_result_ranked_BINS_denovo_ddd.ht')
df=get_binned_models_pd(ht,models=['ae281191'])
df.to_csv(f'{plot_dir}/variant_qc/ae281191_dataframe_plots_june2021.csv', index=False)

2021-06-17 11:00:27 Hail: INFO: Ordering unsorted dataset with network shuffle


In [4]:
snvs=df[df['snv']==True]
indels=df[df['snv']==False]

In [5]:
model="ae281191"
colors={model:'blue'}
metric='n_trans_singletons_synonymous'
data="snvs"
tabs=plot_metric(snvs, metric+" "+data, [metric], y_fun=lambda x: x[0], colors=colors)
data="indels"
tabs2=plot_metric(indels, metric+" "+data, [metric], y_fun=lambda x: x[0], colors=colors)
colors={model:'blue'}
metric='n_untrans_singletons_synonymous'
data="snvs"
tabs3=plot_metric(snvs, metric+" "+data, [metric], y_fun=lambda x: x[0], colors=colors)
data="indels"
tabs4=plot_metric(indels, metric+" "+data, [metric], y_fun=lambda x: x[0], colors=colors)
colors={model:'blue'}
metric='n_trans_singletons_synonymous'
metric2='n_untrans_singletons_synonymous'
data="snvs"
tabs5=plot_metric(snvs, metric+ "/"+metric2+" "+data, [metric,metric2], y_fun=lambda x: x[0]/x[1], colors=colors)
data="indels"
tabs6=plot_metric(indels, metric+" "+data, [metric], y_fun=lambda x:  x[0], colors=colors)

In [6]:
show(tabs5)

In [7]:
df.columns

Index(['rank_id', 'snv', 'singleton', 'trans_singletons',
       'de_novo_high_quality', 'de_novo_medium_quality', 'de_novo_synonymous',
       'bin', 'min_score', 'max_score', 'n', 'n_ins', 'n_del', 'n_ti', 'n_tv',
       'n_1bp_indel', 'n_mod3bp_indel', 'n_singleton',
       'n_high_quality_de_novos', 'n_medium_quality_de_novos',
       'n_high_confidence_de_novos', 'n_de_novo',
       'n_high_quality_de_novos_synonymous', 'n_de_novo_sites',
       'n_trans_singletons_old', 'n_trans_singletons', 'n_untrans_singletons',
       'n_trans_singletons_synonymous',
       'n_trans_singletons_synonymous_original', 'n_untrans_singletons_old',
       'n_untrans_singletons_synonymous',
       'n_untrans_singletons_synonymous_original', 'n_train_trans_singletons',
       'n_omni', 'n_mills', 'n_hapmap', 'n_kgp_high_conf_snvs',
       'fail_hard_filters', 'model'],
      dtype='object')

In [12]:
colors={model:'blue'}
metric='n_trans_singletons_synonymous_original'
metric2='n_untrans_singletons_synonymous_original'
data="snvs"
tabs=plot_metric(snvs, metric+ "/"+metric2+" "+data, [metric,metric2], y_fun=lambda x: x[0]/x[1], colors=colors)

In [13]:
show(tabs)